# Validate with pydantic

In [2]:
from pydantic import BaseModel, ValidationError

# Student is a normal class but is also a Pydantic BaseModel as we inherit from it
class Student(BaseModel):
    name: str 
    gender: str 
    age: int

s1 = Student(name="Beda", gender="F", age=8)
s1

Student(name='Beda', gender='F', age=8)

In [3]:

s1.age = 20
s1.name = "Doddo"
s1

Student(name='Doddo', gender='F', age=20)

In [4]:
try:
    Student(name=2.714, gender=8, age=-8)
except ValidationError as err:
    print(err)

2 validation errors for Student
name
  Input should be a valid string [type=string_type, input_value=2.714, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=8, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [5]:
from pydantic import Field
from typing import Literal

class Student(BaseModel):
    name: str 
    gender: Literal["M", "F"] 
    age: int = Field(gt = -1, lt = 125)

try:
    Student(name="Bibbi", gender="female", age = -1)
except ValidationError as err:
    print(err)

2 validation errors for Student
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='female', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


In [6]:
s2 = Student(name="Bibbi", gender="F", age = "1")
s2

Student(name='Bibbi', gender='F', age=1)

## Serialization and deserialization

- serialization: convert object -> json
- deserialization: convert json -> object

In [7]:
s2.model_dump()

{'name': 'Bibbi', 'gender': 'F', 'age': 1}

In [8]:
import json
with open('housing.json', 'r') as file:
    json_data = json.loads(file.read())

json_data

[{'address': 'Sveavägen 98, Norrmalm',
  'price': 5200000,
  'area': 34,
  'rooms': 1.5},
 {'address': 'Hornsgatan 72, Södermalm',
  'price': 7600000,
  'area': 52,
  'rooms': 2},
 {'address': 'Värtavägen 12, Östermalm',
  'price': 11200000,
  'area': 78,
  'rooms': 3},
 {'address': 'Götgatan 112, Södermalm',
  'price': 8900000,
  'area': 65,
  'rooms': 2.5},
 {'address': 'Hälsingegatan 4, Vasastan',
  'price': 6700000,
  'area': 44,
  'rooms': 1.5}]

In [9]:
from pydantic import BaseModel, Field
class House(BaseModel):
    address: str
    price: int = Field(lt = 10_000_000)
    area: float
    rooms: float

In [10]:
House(address="....", price=8391, area=32, rooms=2.0)

House(address='....', price=8391, area=32.0, rooms=2.0)

In [11]:
json_data[0]

{'address': 'Sveavägen 98, Norrmalm',
 'price': 5200000,
 'area': 34,
 'rooms': 1.5}

In [12]:
# House(address="....", price=5200..., area=34.0, rooms=1.5)
House(**json_data[0])

House(address='Sveavägen 98, Norrmalm', price=5200000, area=34.0, rooms=1.5)

In [ ]:
houses = []
for house_data in json_data:
    try: 
        houses.append(House(**house_data))
    except ValidationError:
        None 

houses

[House(address='Sveavägen 98, Norrmalm', price=5200000, area=34.0, rooms=1.5),
 House(address='Hornsgatan 72, Södermalm', price=7600000, area=52.0, rooms=2.0),
 House(address='Götgatan 112, Södermalm', price=8900000, area=65.0, rooms=2.5),
 House(address='Hälsingegatan 4, Vasastan', price=6700000, area=44.0, rooms=1.5)]

In [14]:
from pydantic import BaseModel

class House(BaseModel):
    address: str
    price: int
    area: float
    rooms: float

[House(**item) for item in json_data]

[House(address='Sveavägen 98, Norrmalm', price=5200000, area=34.0, rooms=1.5),
 House(address='Hornsgatan 72, Södermalm', price=7600000, area=52.0, rooms=2.0),
 House(address='Värtavägen 12, Östermalm', price=11200000, area=78.0, rooms=3.0),
 House(address='Götgatan 112, Södermalm', price=8900000, area=65.0, rooms=2.5),
 House(address='Hälsingegatan 4, Vasastan', price=6700000, area=44.0, rooms=1.5)]